### Columnas de Floor

In [145]:
import pandas as pd
import numpy as np
import re 

In [146]:
data_location = "./properati.csv"
data = pd.read_csv(data_location, sep=",")
print(data.dtypes)
data.head(3)

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [147]:
data.shape

(121220, 26)

In [148]:
#Creación de una nueva columna con la copia de los valores de la columna 'floor'
data['floor_copia'] = data['floor']
data['property_type_copia'] = data['property_type']
#Cambio a minúscula estas dos nuevas columnas creadas
data['description_copia'] = data['description'].astype(str).str.lower()
data['title_copia']= data['title'].astype(str).str.lower()

In [149]:
#Comprobación de valores null en data 'floor_copia'
data['floor_copia'].isnull().value_counts()

True     113321
False      7899
Name: floor_copia, dtype: int64

In [150]:
#Comprobación de valores null en data 'description_copia'
data['description'].isnull().value_counts()

False    121218
True          2
Name: description, dtype: int64

In [151]:
#Veo qué valores son los que faltan en 'description_copia'
data[data['description'].isnull()]

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,rooms,expenses,properati_url,description,title,image_thumbnail,floor_copia,property_type_copia,description_copia,title_copia
10664,10664,sell,apartment,Villa Carlos Paz,|Argentina|Córdoba|Villa Carlos Paz|,Argentina,Córdoba,3832791.0,"-31.4207828,-64.4992141",-31.420783,...,NaN,NaN,http://www.properati.com.ar/168nc_venta_depart...,NaN,HERMOSO DEPARTAMENTO CON PISCINA Y COCHERA,https://thumbs4.properati.com/6/JiAoy6zsKPeBob...,NaN,apartment,nan,hermoso departamento con piscina y cochera
96691,96691,sell,apartment,San Fernando,|Argentina|Bs.As. G.B.A. Zona Norte|San Fernando|,Argentina,Bs.As. G.B.A. Zona Norte,3429088.0,NaN,NaN,...,NaN,NaN,http://www.properati.com.ar/1bj1r_venta_depart...,NaN,Departamento - San Fernando,https://thumbs4.properati.com/1/FZZcNCsb4-ucU2...,NaN,apartment,nan,departamento - san fernando


In [152]:
#Categrias en property_type y qué cantidaa hay
data['property_type_copia'].value_counts()

apartment    71065
house        40268
PH            5751
store         4136
Name: property_type_copia, dtype: int64

In [153]:
#agrupo por tipo de propiedad
tipo_propiedad = data.groupby('property_type_copia')

In [154]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 100, 5000]
pisos = pd.cut(data.floor_copia, bins, right = False)

In [155]:
pisos.value_counts()

[1, 2)         2080
[2, 3)         1542
[3, 4)          914
[9, 15)         743
[4, 5)          658
[5, 6)          484
[6, 7)          483
[7, 8)          355
[8, 9)          248
[15, 100)       224
[100, 5000)     168
[0, 1)            0
Name: floor_copia, dtype: int64

In [171]:
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

property_type_copia,PH,apartment,house,store
floor_copia,,,,
"[0, 1)",0,0,0,0
"[1, 2)",154,1517,385,24
"[2, 3)",146,1096,290,10
"[3, 4)",63,833,18,0
"[4, 5)",31,625,1,1
"[5, 6)",38,444,2,0
"[6, 7)",42,439,2,0
"[7, 8)",31,320,4,0
"[8, 9)",12,232,4,0


Observamos valores incorrectos como house o PH con más de 15 pisos. 
Invesigamos por qué había tantos pisos en esas categorías y llegamos a la conclusión de que habia PH o house mal clasificados como tal. En otros casos, los pisos estaban mal imputados.


In [157]:
data[(data.property_type_copia == 'PH') & (data.floor_copia == 2 )].sample(10)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,rooms,expenses,properati_url,description,title,image_thumbnail,floor_copia,property_type_copia,description_copia,title_copia
41102,41102,sell,PH,Villa Dominico,|Argentina|Bs.As. G.B.A. Zona Sur|Avellaneda|V...,Argentina,Bs.As. G.B.A. Zona Sur,3427447.0,"-34.690966,-58.318919",-34.690966,...,3.0,NaN,http://www.properati.com.ar/18tdc_venta_ph_vil...,Corredor Responsable: Martin Veltri - C.M.Q 80...,VENTA PH 3 AMBIENTES CON TERRAZA,https://thumbs4.properati.com/0/-ob0WylGbvcESX...,2.0,PH,corredor responsable: martin veltri - c.m.q 80...,venta ph 3 ambientes con terraza
34278,34278,sell,PH,Córdoba,|Argentina|Córdoba|Córdoba|,Argentina,Córdoba,3860259.0,"-31.4062519,-64.2375441",-31.406252,...,2.0,NaN,http://www.properati.com.ar/18bkg_venta_ph_cor...,Duplex en Venta de 2 dorm. en Cordoba,Venta en pozo de 4 Duplex en Housing,https://thumbs4.properati.com/6/lGbU1ZB7W_27bz...,2.0,PH,duplex en venta de 2 dorm. en cordoba,venta en pozo de 4 duplex en housing
12976,12976,sell,PH,Córdoba,|Argentina|Córdoba|Córdoba|,Argentina,Córdoba,3860259.0,"-31.4585852,-64.162674",-31.458585,...,3.0,NaN,http://www.properati.com.ar/16nr7_venta_ph_cor...,Duplex en Venta de 3 dorm. en Cordoba,+ CLAROS DEL BOSQUE - ESQUINA FRENTE A E,https://thumbs4.properati.com/9/3BtDr9Me8kLCsu...,2.0,PH,duplex en venta de 3 dorm. en cordoba,+ claros del bosque - esquina frente a e
29760,29760,sell,PH,Córdoba,|Argentina|Córdoba|Córdoba|,Argentina,Córdoba,3860259.0,NaN,NaN,...,2.0,NaN,http://www.properati.com.ar/17zvl_venta_ph_cor...,MAITEN - VENTA - DEPARTAMENTO - 1 DORMITORIO -...,PH EN VENTA,https://thumbs4.properati.com/3/MyTM-IpyPj0Lja...,2.0,PH,maiten - venta - departamento - 1 dormitorio -...,ph en venta
23454,23454,sell,PH,Lanús,|Argentina|Bs.As. G.B.A. Zona Sur|Lanús|Lanús|,Argentina,Bs.As. G.B.A. Zona Sur,NaN,"-34.700431,-58.398685",-34.700431,...,3.0,NaN,http://www.properati.com.ar/17lnx_venta_ph_lan...,Corredor Responsable: FRANCISCO D'ATRI - CPMCA...,Ph al frente 3 amb c/cochera,https://thumbs4.properati.com/4/tfZCFUEA6iMuRG...,2.0,PH,corredor responsable: francisco d'atri - cpmca...,ph al frente 3 amb c/cochera
40705,40705,sell,PH,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436080.0,"-34.468726,-58.5577",-34.468726,...,3.0,NaN,http://www.properati.com.ar/18s13_venta_ph_bec...,Corredor Responsable: Uno Bienes Raices SRL - ...,Duplex en venta San Isidro,https://thumbs4.properati.com/5/c26fMYOBsnWpbU...,2.0,PH,corredor responsable: uno bienes raices srl - ...,duplex en venta san isidro
7708,7708,sell,PH,Caballito,|Argentina|Capital Federal|Caballito|,Argentina,Capital Federal,3435874.0,"-34.6154989,-58.4637207",-34.615499,...,3.0,1800.0,http://www.properati.com.ar/15ytj_venta_ph_cab...,"3 AMBIENTES CON BALCÓN AL FRENTE, EN VENTA, EN...",PH EN VENTA,https://thumbs4.properati.com/8/kJph1RysBTZuBj...,2.0,PH,"3 ambientes con balcón al frente, en venta, en...",ph en venta
1476,1476,sell,PH,Córdoba,|Argentina|Córdoba|Córdoba|,Argentina,Córdoba,3860259.0,"-31.340828,-64.2839297",-31.340828,...,3.0,NaN,http://www.properati.com.ar/15idu_venta_ph_cor...,Duplex en Venta de 3 dorm. en Cordoba,Duplex en Venta de 3 dorm. en Cordoba,https://thumbs4.properati.com/0/W6MdbstZHkCCM5...,2.0,PH,duplex en venta de 3 dorm. en cordoba,duplex en venta de 3 dorm. en cordoba
32381,32381,sell,PH,Villa Devoto,|Argentina|Capital Federal|Villa Devoto|,Argentina,Capital Federal,3427451.0,"-34.591624,-58.51921",-34.591624,...,1.0,NaN,http://www.properati.com.ar/186wi_venta_ph_vil...,Corredor Responsable: Uno Bienes Raices SRL - ...,MONOAMBIENTE NO ES APTO CRÉDITO JARDÍN Y PARRILLA,https://thumbs4.properati.com/0/-CSJHwdJZDY8wP...,2.0,PH,corredor responsable: uno bienes raices srl - ...,monoambiente no es apto crédito jardín y parrilla
61021,61021,sell,PH,Pinamar,|Argentina|Buenos Aires Costa Atlántica|Pinamar|,Argentina,Buen

In [158]:
data.loc[39435, 'description_copia']

'hermoso ph de 3 ambientes en venta!!!único en el piso. sin expensas!!!!!2º piso por escalera.excelente ubicación a media cuadra de av. córdoba y a 4 cuadras del metrobus y del tren san martín.se ingresa a un amplio living comedor apaisado con salida al balcón corrido, dos dormitorios con placard, generosa cocina con comedor diario de 4,40 x 2,7 y lavadero independiente.todo externo y muy luminoso. pisos de marmol y parquet excelente estado!venta no condicionada y apto crédito hipotecario.sucursal almagro sucursal villa crespo 4857- deja constancia que los m2 indicados son aproximados, al igual que las medidas parciales de los ambientes, y el valor consignado de expensas mensuales está sujeto a verificación y/o ajustes. fotos de carácter no contractual.� -  - ebertolotti'

Procedemos a limpiar esos valores mal imputados en los PH

In [127]:
#maskPH = (data.property_type_copia == 'PH') & (data.floor_copia > 1)

In [128]:
#data.loc[maskPH, 'property_type_copia'] = 'apartment'

Comprobamos que hayamos imputado correctamente:

In [129]:
#data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

property_type_copia,PH,apartment,house,store
floor_copia,,,,
"[0, 1)",0,0,0,0
"[1, 2)",154,1517,385,24
"[2, 3)",0,1242,290,10
"[3, 4)",0,896,18,0
"[4, 5)",0,656,1,1
"[5, 6)",0,482,2,0
"[6, 7)",0,481,2,0
"[7, 8)",0,351,4,0
"[8, 9)",0,244,4,0


In [161]:
patron = "(?P<numero>\d+)[\º|°]?\s*piso"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()

In [162]:
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero'))

In [163]:
data['floor_copia'].isnull().value_counts()

True     103365
False     17855
Name: floor_copia, dtype: int64

In [173]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 100, 5000]
pisos = pd.cut(data.floor_copia, bins, right = False)
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

TypeError: '<' not supported between instances of 'float' and 'str'

In [140]:
patron = "(?P<numero>\d+)[\w*]?\spiso"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero'))

In [141]:
data['floor_copia'].isnull().value_counts()

True     97959
False    23261
Name: floor_copia, dtype: int64

In [102]:
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

property_type_copia,PH,apartment,house,store
floor_copia,,,,
"[0, 1)",0,0,0,0
"[1, 2)",154,1517,385,24
"[2, 3)",146,1096,290,10
"[3, 4)",63,833,18,0
"[4, 5)",31,625,1,1
"[5, 6)",38,444,2,0
"[6, 7)",42,439,2,0
"[7, 8)",31,320,4,0
"[8, 9)",12,232,4,0


In [134]:
patron = "(?P<numero>\d+)[\s]?\w*[\s]?piso"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero'))

In [135]:
data['floor_copia'].isnull().value_counts()

True     100433
False     20787
Name: floor_copia, dtype: int64

In [136]:
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

property_type_copia,PH,apartment,house,store
floor_copia,,,,
"[0, 1)",0,0,0,0
"[1, 2)",154,1517,385,24
"[2, 3)",0,1242,290,10
"[3, 4)",0,896,18,0
"[4, 5)",0,656,1,1
"[5, 6)",0,482,2,0
"[6, 7)",0,481,2,0
"[7, 8)",0,351,4,0
"[8, 9)",0,244,4,0


In [142]:
patron = "piso\w*\s*(?P<numero>\d+)"

busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero'))

In [143]:
data['floor_copia'].isnull().value_counts()

True     97959
False    23261
Name: floor_copia, dtype: int64

In [144]:
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

property_type_copia,PH,apartment,house,store
floor_copia,,,,
"[0, 1)",0,0,0,0
"[1, 2)",154,1517,385,24
"[2, 3)",0,1242,290,10
"[3, 4)",0,896,18,0
"[4, 5)",0,656,1,1
"[5, 6)",0,482,2,0
"[6, 7)",0,481,2,0
"[7, 8)",0,351,4,0
"[8, 9)",0,244,4,0
